In [1]:
!pip install pyura
!pip install geopy

# Import packages
import datetime
import math
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
import time
import zipfile
import requests
import io
from datetime import date
from pyura import Client
from requests import Session
from geopy.geocoders import Nominatim

# URA Access Key
access_key = '8c04ea68-08ad-4e37-ae9d-27a704d979b6'


# 1. Get Data from all sources / api

In [2]:
def get_json_from_zip_url(json_name, zip_file_url):
    r = requests.get(zip_file_url, stream=True)
    with zipfile.ZipFile(io.BytesIO(r.content)) as myzip:
        with myzip.open(json_name) as myfile:
            data = json.load(myfile)
            return data
    return None

def get_df_from_url(xl_url):
    return pd.read_csv(xl_url)

json_name = 'hawker-centres-geojson.geojson'
zip_file_url = 'https://data.gov.sg/dataset/aeaf4704-5be1-4b33-993d-c70d8dcc943e/download'
data = get_json_from_zip_url(json_name, zip_file_url)

In [3]:
hawker_dict_data_gov = {}
for i in range(len(data['features'])):
    string = (data['features'][i]['properties']['Description'])
    list_of_strings = string.split()
    postal_code = list_of_strings[list_of_strings.index('<th>ADDRESSPOSTALCODE</th>') + 1][4:6]
    address = list_of_strings[list_of_strings.index('<th>NAME</th>') + 1 : list_of_strings.index('<th>ADDRESSTYPE</th>') - 2]
    hawker_add = " ".join(address)[4:-5]
    hawker_dict_data_gov[postal_code] = hawker_add
    
hawker_dict_data_gov

{'76': 'Yishun Ring Road Blk 104/105 (Chong Pang Market and Food Centre)',
 '61': 'Taman Jurong Market and Food Centre',
 '38': 'Aljunied Ave 2 Blk 117 (Blk 117 Aljunied Market and Food Centre)',
 '15': 'Jalan Bukit Merah Blk 6 (ABC Brickworks Market/Food Centre)',
 '52': 'Our Tampines Hub',
 '60': 'Jurong East Street 24 Blk 254 (Yuhua Village Market and Food Centre)',
 '27': 'Ghim Moh Road Blk 20',
 '57': 'Sembawang Hills Food Centre (Jalan Leban Food Centre)',
 '05': "New Market Road Blk 32 (People's Park Food Centre)",
 '14': 'Mei Chin Road Blk 159 (Mei Chin Road Market)',
 '08': 'Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Pagar Plaza Market and Food Centre)',
 '10': 'Telok Blangah Drive Blk 79 (Telok Blangah Food Centre)',
 '16': 'Havelock Road Blk 22A/B (Havelock Road Cooked Food Centre)',
 '46': 'East Coast Lagoon Food Village',
 '33': 'Geylang Bahru Blk 69 (Blk 69 Geylang Bahru Market and Food Centre)',
 '64': 'Jurong West Street 52 Blk 505',
 '67': 'Bukit Panjang Hawker Centre',


# 1.2 Get 5Y Data from URA

In [4]:
def ura_to_df(json_data):
    main_df = pd.DataFrame()
    for i in range(len(json_data)):
        trans_list = json_data[i]
        df = pd.DataFrame(trans_list)
        formatted_df = pd.concat([df.drop('transaction', axis=1), pd.DataFrame(df['transaction'].tolist())], axis=1)
        # Append to df 
        main_df = main_df.append(formatted_df)
    return main_df

def query_ura(access_key):
    main_df = pd.DataFrame()
    # Gen Daily Tokenb
    sess = Session()
    response = sess.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers={"AccessKey": access_key})
    token = response.json()["Result"]
    # GET PRIVATE RESI DATA IN BATCHES (4) 
    for batch in range(1,5):
        print(f"Querying URA API Batch {batch}/4")
        out = sess.get("https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction", params={"batch": batch}, headers={"AccessKey": access_key, "Token": token})
        response_json = out.json()
        if "Result" in response_json:
            response_data = response_json["Result"]
        else:
            response_data = response_json
        formatted_df = ura_to_df(response_data)
        main_df = main_df.append(formatted_df)
    main_df = main_df.reset_index(drop=True)
    return main_df

In [5]:
sess = Session()
response = sess.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers={"AccessKey": access_key})

ura_df = query_ura(access_key)
ura_df.to_csv('5y_ura.csv')

Querying URA API Batch 1/4
Querying URA API Batch 2/4
Querying URA API Batch 3/4
Querying URA API Batch 4/4


# 1.3 Get all amenities in each district

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pathlib import Path
import time
import shapefile
import os
import shutil
import zipfile

# Global path for the scripts
path_env = os.path.dirname(os.path.abspath("bt_4222_main.ipynb"))

# Common paths used by multiple scripts
path_chromedriver = str(Path(path_env, 'chromedriver.exe'))
path_download = os.path.join(Path.home(), "Downloads")

# Map to Specific Location in the District for OneMap Search
district_mapping = {
    "1" : ["RAFFLES PLACE MRT STATION EXIT A", "CECIL BUILDING", "MARINA BAY SINGAPORE", "PEOPLE'S PARK COMPLEX"],
    "2" : ["ANSON ROAD", "TANJONG PAGAR MRT STATION EXIT A"],
    "3" : ["QUEENSTOWN MRT STATION EXIT A", "TIONG BAHRU MRT STATION EXIT A"],
    "4" : ["TELOK BLANGAH RISE MARKET", "HARBOURFRONT MRT STATION EXIT A"],
    "5" : ["PASIR PANJANG MRT STATION EXIT A", "CLEMENTI MRT STATION EXIT A"],
    "6" : ["HIGH STREET PLAZA", "CALTEX BEACH ROAD"],
    "7" : ["DBS MIDDLE ROAD 210", "GOLDEN MILE COMPLEX"],
    "8" : ["LITTLE INDIA MRT STATION EXIT A"],
    "9" : ["ORCHARD MRT STATION EXIT A", "THE CAIRNHILL", "GREAT WORLD MRT STATION"],
    "10" : ["BUKIT TIMAH ROAD", "HOLLAND VILLAGE MRT STATION", "TANGLIN MALL"],
    "11" : ["WATTEN ESTATE", "NOVENA MRT STATION EXIT A", "THOMSON PLAZA"],
    "12" : ["BALESTIER PLAZA", "TOA PAYOH MRT STATION EXIT A", "SERANGOON MRT STATION EXIT A"],
    "13" : ["MACPHERSON MRT STATION EXIT A", "BRADDELL MRT STATION EXIT A"],
    "14" : ["PAYA LEBAR MRT STATION EXIT A", "EUNOS MRT STATION EXIT A"],
    "15" : ["TANJONG KATONG GIRLS' SCHOOL", "KOON SENG PARK", "TANJONG KATONG MRT STATION EXIT A"],
    "16" : ["BEDOK MRT STATION EXIT A", "TANAH MERAH MRT STATION EXIT A", "SUNGEI BEDOK MRT STATION", "KEW DRIVE PLAYGROUND"],
    "17" : ["MARIAM WAY PLAYGROUND", "KEMBANGAN MRT STATION EXIT A"],
    "18" : ["TAMPINES MRT STATION EXIT A", "PASIR RIS MRT STATIONEXIT A"],
    "19" : ["SERANGOON GARDEN MARKET EXIT A", "HOUGANG MRT STATION EXIT A", "PUNGGOL MRT STATION EXIT A"],
    "20" : ["BISHAN MRT STATION EXIT A", "ANG MO KIO MRT STATION EXIT A"],
    "21" : ["ESSO UPPER BUKIT TIMAH A", "CLEMENTI PARK", "ULU PANDAN COMMUNITY CLUB"],
    "22" : ["JURONG EAST MRT STATION EXIT A", "BOON LAY MRT STATION EXIT A", "LAKESIDE MRT STATION EXIT A", "CHINESE GARDEN MRT STATION EXIT A"],
    "23" : ["HILLVIEW MRT STATION EXIT A", "GERMAN EUROPEAN SCHOOL SINGAPORE", "BUKIT PANJANG MRT STATION EXIT A1", "CHOA CHU KANG MRT STATION EXIT A"],
    "24" : ["LIM CHU KANG CAMP I", "GARDEN VALE @ TENGAH"],
    "25" : ["KRANJI MRT STATION EXIT A", "THE WOODGROVE"],
    "26" : ["UPPER THOMSON MRT STATION", "SPRINGLEAF GARDEN"],
    "27" : ["YISHUN MRT STATION EXIT A", "SEMBAWANG MRT STATION EXIT A"],
    "28" : ["THE SELETAR MALL"]
}

district_postal = {
    "1" : ["01", "02", "03", "04", "05", "06"],
    "2" : ["07", "08"],
    "3" : ["14", "15", "16"],
    "4" : ["09", "10"],
    "5" : ["11", "12", '13'],
    "6" : ["17"],
    "7" : ["18", "19"],
    "8" : ["20", "21"],
    "9" : ["22", "23"],
    "10" : ["24", "25", "26", "27"],
    "11" : ["28", "29", "30"],
    "12" : ["31", "32", "33"],
    "13" : ["34", "35", "36", "37"],
    "14" : ["38", "39", "40", "41"],
    "15" : ["42", "43", "44", "45"],
    "16" : ["46", "47", "48"],
    "17" : ["49", "50", "81"],
    "18" : ["51", "52"],
    "19" : ["53", "54", "55", "82"],
    "20" : ["56", "57"],
    "21" : ["58", "59"],
    "22" : ["60", "61", "62", "63", "64"],
    "23" : ["65", "66", "67", "68"],
    "24" : ["69", "70", "71"],
    "25" : ["72", "73"],
    "26" : ["77", "78"],
    "27" : ["75", "76"],
    "28" : ["79", "80"]
}



def get_dict_data(address, driver):
    # address is list
    ans = dict()
    count = 0
    # Just download shapefiles (Only need run once / Any address)
    download_all_shp(address[0], driver)
    os.chdir(path_download)
    for add in address:
        count += 1
        print(f"{add}, {count}/{len(address)}")
        school = download_all_schools(str(add), driver)
        inter = dict()
        inter['school'] = school
        for i in ['hawkercentre', 'supermarkets']:
            try:
                with zipfile.ZipFile(f"{i}.zip","r") as zip_ref:
                    zip_ref.extractall(f"{i}")
                sf = shapefile.Reader(f"{i}/{i}/{i.upper()}.shp")
                output = sf.__geo_interface__
                inter[i] = output['features']
                sf.close()
            except:
                inter[i] = None 
        ans[add] = inter
    try:
        for i in ['hawkercentre', 'supermarkets']:
            os.remove(f"{i}.zip")
            shutil.rmtree(f"{i}")
    except FileNotFoundError:
        pass
    os.chdir(path_env)
    return ans

def download_all_shp(address, driver):
    url = 'https://www.onemap.sg'
    driver.get(url)
    try:
        driver.find_element_by_xpath('//*[@id="my-text"]/img').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="search-text"]').clear()
        query = driver.find_element_by_xpath('//*[@id="search-text"]')
    except:
        try:
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="search-text"]').clear()
            query = driver.find_element_by_xpath('//*[@id="search-text"]')
        except:
            pass
    query.send_keys(address)
    time.sleep(5)
    query.send_keys(Keys.ENTER)
    # Essential Amenities
    driver.find_element_by_xpath('//*[@id="EssDisplay"]').click()
    time.sleep(1)
    for i in [1, 2]:
        driver.find_element_by_xpath(f'//*[@id="mCSB_4_container"]/div/div[{str(i)}]/div/div[2]/label').click()
        select = Select(driver.find_element_by_xpath('/html/body/div[2]/div/select'))
        # select by value 
        select.select_by_value('shp')
        driver.find_element_by_xpath('/html/body/div[2]/div/div[10]/button[2]').click()
        driver.find_element_by_xpath('/html/body/div[2]/div/div[10]/button[1]').click()
    return None

def download_all_schools(address, driver):
    url = 'https://www.onemap.sg'
    driver.get(url)
    try:
        driver.find_element_by_xpath('//*[@id="my-text"]/img').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="search-text"]').clear()
        query = driver.find_element_by_xpath('//*[@id="search-text"]')
    except:
        try:
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="search-text"]').clear()
            query = driver.find_element_by_xpath('//*[@id="search-text"]')
        except:
            pass
    query.send_keys(address)
    time.sleep(5)
    query.send_keys(Keys.ENTER)
    # Essential Amenities
    driver.find_element_by_xpath('//*[@id="EssDisplay"]').click()
    time.sleep(1)
    try:
        #Schools
        driver.find_element_by_xpath('//*[@id="SQDisplay"]').click()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="schoolMenu"]/button[2]').click()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="AgreeBoxSQ"]/label').click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        schools = [school.text for school in soup.find_all('span', class_='school-header ng-binding')]
    except Exception as e:
        print(e)
        schools = None
    return schools

# Get distance between 2 locations using claims distance finder https://pacgov.agd.gov.sg/ipac/html/W51RouteDistanceFinder.html
def get_distance(start, end, driver):
    url = 'https://pacgov.agd.gov.sg/ipac/html/W51RouteDistanceFinder.html'
    driver.get(url)
    driver.find_element_by_xpath('//*[@id="txtSearchFromText"]').send_keys(str(start))
    driver.find_element_by_xpath('//*[@id="txtSearchToText"]').send_keys(str(end))
    driver.find_element_by_xpath('/html/body/form/table[1]/tbody/tr[12]/td[3]/input').click()
    driver.find_element_by_xpath('//*[@id="divResults"]/table/tbody/tr[2]/td/div').click()
    driver.find_element_by_xpath('/html/body/form/table[1]/tbody/tr[14]/td[3]/input').click()
    driver.find_element_by_xpath('//*[@id="divResults"]/table/tbody/tr[2]/td/div').click()
    driver.find_element_by_xpath('/html/body/form/table[1]/tbody/tr[16]/td/input').click()
    
def get_2km_nearby(list_of_address, district_mapping, district_postal, hawker_gov):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    final = get_dict_data(list_of_address, driver)
    main_df = pd.DataFrame()
    for address, contents in final.items():
        new_dict = {}
        new_dict['address'] = address
        for key, value in district_mapping.items():
            if address in value:
                f_key = key
        postal_codes = district_postal[f_key]
        for tag, content in contents.items():
            if tag == 'school':
                new_dict[tag] = content
            else:
                # Form new list 
                all_names = []
                # From SHP Downloaded
                if content is not None:
                    for place in content:
                        if tag == 'hawkercentre':
                            if place['properties']['ADDRESSPOS'][:2] in postal_codes:
                                name = place['properties']['NAME']
                                if name in all_names:
                                    continue
                                else:
                                    all_names.append(name)
                        elif tag == 'supermarkets':
                            if place['properties']['POSTCODE'][:2] in postal_codes:
                                name = place['properties']['LIC_NAME']
                                if name in all_names:
                                    continue
                                else:
                                    all_names.append(name)
                # Hawker Centres from Data.gov
                if tag == 'hawkercentre':
                    for keys in hawker_gov.keys():
                        if keys in postal_codes:
                            if hawker_gov[keys] not in all_names: 
                                all_names.append(hawker_gov[keys])
                new_dict[tag] = all_names
        data = [[new_dict['address'], new_dict['school'], new_dict['hawkercentre'], new_dict['supermarkets']]]
        df = pd.DataFrame(data, columns = ['address', 'school', 'hawkercentre', 'supermarkets'])
        main_df = main_df.append(df)
    driver.close()
    return main_df

In [7]:
flat_list = [item for sublist in district_mapping.values() for item in sublist]
nearby_df = get_2km_nearby(flat_list, district_mapping, district_postal, hawker_dict_data_gov)
nearby_df.head()

RAFFLES PLACE MRT STATION EXIT A, 1/69
CECIL BUILDING, 2/69
MARINA BAY SINGAPORE, 3/69
PEOPLE'S PARK COMPLEX, 4/69
ANSON ROAD, 5/69
TANJONG PAGAR MRT STATION EXIT A, 6/69
QUEENSTOWN MRT STATION EXIT A, 7/69
TIONG BAHRU MRT STATION EXIT A, 8/69
TELOK BLANGAH RISE MARKET, 9/69
HARBOURFRONT MRT STATION EXIT A, 10/69
PASIR PANJANG MRT STATION EXIT A, 11/69
CLEMENTI MRT STATION EXIT A, 12/69
HIGH STREET PLAZA, 13/69
CALTEX BEACH ROAD, 14/69
DBS MIDDLE ROAD 210, 15/69
GOLDEN MILE COMPLEX, 16/69
LITTLE INDIA MRT STATION EXIT A, 17/69
ORCHARD MRT STATION EXIT A, 18/69
THE CAIRNHILL, 19/69
GREAT WORLD MRT STATION, 20/69
BUKIT TIMAH ROAD, 21/69
HOLLAND VILLAGE MRT STATION, 22/69
TANGLIN MALL, 23/69
WATTEN ESTATE, 24/69
NOVENA MRT STATION EXIT A, 25/69
THOMSON PLAZA, 26/69
BALESTIER PLAZA, 27/69
TOA PAYOH MRT STATION EXIT A, 28/69
SERANGOON MRT STATION EXIT A, 29/69
MACPHERSON MRT STATION EXIT A, 30/69
BRADDELL MRT STATION EXIT A, 31/69
PAYA LEBAR MRT STATION EXIT A, 32/69
EUNOS MRT STATION EXIT 

,address,school,hawkercentre,supermarkets
0,RAFFLES PLACE MRT STATION EXIT A,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BENDEMEER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,CECIL BUILDING,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BUKIT MER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,MARINA BAY SINGAPORE,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BENDEMEER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,PEOPLE'S PARK COMPLEX,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BENDEMEER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,ANSON ROAD,"[BUKIT MERAH SECONDARY SCHOOL, CHIJ ST. THERES...",[Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Paga...,[]


In [8]:
ameneties_df = pd.DataFrame()
for key, value in district_mapping.items():
    school_list = []
    hawker_list = []
    market_list = []
    for address in value:
        try:
            school_list += nearby_df.loc[nearby_df.address == address, 'school'].values[0]
        except TypeError:
            continue
        try:
            hawker_list += nearby_df.loc[nearby_df.address == address, 'hawkercentre'].values[0]
        except TypeError:
            continue
        try:
            market_list += nearby_df.loc[nearby_df.address == address, 'supermarkets'].values[0]
        except TypeError:
            continue
    data = [[key, list(set(school_list)), list(set(hawker_list)), list(set(market_list))]]
    df = pd.DataFrame(data, columns = ['district', 'school', 'hawkercentre', 'supermarkets'])
    ameneties_df = ameneties_df.append(df)
ameneties_df.head()

,district,school,hawkercentre,supermarkets
0,1,"[OUTRAM SECONDARY SCHOOL, BENDEMEER SECONDARY ...",[Amoy Street Food Centre (Telok Ayer Food Cent...,[]
0,2,"[OUTRAM SECONDARY SCHOOL, CRESCENT GIRLS' SCHO...",[Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Paga...,[]
0,3,"[OUTRAM SECONDARY SCHOOL, ST. MARGARET'S SECON...","[Commonwealth Crescent Market, Bukit Merah Lan...","[NTUC FAIRPRICE CO-OPERATIVE LTD, COLD STORAGE..."
0,4,"[OUTRAM SECONDARY SCHOOL, QUEENSWAY SECONDARY ...",[Telok Blangah Drive Blk 79 (Telok Blangah Foo...,"[THE QUALITYMART DEPOT PTE. LTD., NTUC FAIRPRI..."
0,5,"[ST. MARGARET'S SECONDARY SCHOOL, NUS HIGH SCH...","[Clementi West Street 2 Blk 726, West Coast Dr...","[NTUC FAIRPRICE CO-OPERATIVE LTD, TANESEI TRAD..."


In [9]:
ameneties_df.to_csv('ameneties_in_district.csv')

# 1.4 Append Geo Coords for Transactions

In [10]:
hdb_df = pd.read_csv('https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/resale_flat_prices_based_on_registration_date_from_jan_2017_onwards.csv')

In [11]:
# FOR HDB DATAFRAME

def replace_words(x, words):
    s = x['street']
    for k, v in words.items():
        if k in s:
            # Just for road - 
            if (k == 'RD') and ("ORD" in s):
                break
            s = s.replace(k, v)
    x['street'] = s
    return x

naming_mapping = {
    'CTRL' : 'CENTRAL',
    'UPP': 'UPPER',
    'JLN': 'JALAN',
    'TG': 'TANJONG',
    'BT': 'BUKIT',
    'NTH': 'NORTH',
    'LOR': 'LORONG', 
    'RD' : 'ROAD',
    'KG' : 'KAMPONG',
    'C\'WEALTH' : 'COMMONWEALTH'
}

hdb_df['street'] = hdb_df['street_name']
hdb_df = hdb_df.apply(lambda x: replace_words(x, naming_mapping), axis=1)

In [12]:
list_streets = list(ura_df.street.unique()) + list(hdb_df.street.unique())

In [13]:
# Requires good internet connection to run, if cant run please use existing csv

lat_long_mapping = {}
# User Agent can be anything 
geolocator = Nominatim(user_agent="newtestuserbtproj")
count = 0
invalid_count = 0
total = len(list_streets)
for street in list_streets:
    try:
        print(f"{count} / {total}")
        location = geolocator.geocode({"street": street, "country": "Singapore"})
        lat_long_mapping[street] = [location.latitude, location.longitude]
        count += 1
        #print(f"Qeurying {count} / {total} unique addresses from Nominatim API")
    except AttributeError:
        invalid_count += 1
        print(f"{street}, {count} / {total}")
        continue
print(invalid_count)

0 / 2221
1 / 2221
2 / 2221
3 / 2221
4 / 2221
5 / 2221
6 / 2221
7 / 2221
8 / 2221
9 / 2221
10 / 2221
11 / 2221
12 / 2221
13 / 2221
14 / 2221
15 / 2221
16 / 2221
17 / 2221
18 / 2221
19 / 2221
20 / 2221
21 / 2221
22 / 2221
23 / 2221
24 / 2221
25 / 2221
26 / 2221
27 / 2221
28 / 2221
29 / 2221
30 / 2221
31 / 2221
32 / 2221
33 / 2221
34 / 2221
35 / 2221
36 / 2221
37 / 2221
38 / 2221
39 / 2221
40 / 2221
41 / 2221
42 / 2221
43 / 2221
44 / 2221
45 / 2221
46 / 2221
47 / 2221
48 / 2221
49 / 2221
50 / 2221
51 / 2221
52 / 2221
53 / 2221
54 / 2221
55 / 2221
56 / 2221
57 / 2221
58 / 2221
59 / 2221
PARK ROAD, 59 / 2221
59 / 2221
60 / 2221
61 / 2221
62 / 2221
63 / 2221
64 / 2221
65 / 2221
66 / 2221
67 / 2221
68 / 2221
69 / 2221
70 / 2221
71 / 2221
72 / 2221
73 / 2221
74 / 2221
75 / 2221
76 / 2221
77 / 2221
78 / 2221
79 / 2221
80 / 2221
81 / 2221
82 / 2221
83 / 2221
84 / 2221
85 / 2221
86 / 2221
87 / 2221
88 / 2221
89 / 2221
90 / 2221
91 / 2221
92 / 2221
93 / 2221
94 / 2221
95 / 2221
96 / 2221
97 / 2221

728 / 2221
729 / 2221
730 / 2221
731 / 2221
732 / 2221
733 / 2221
734 / 2221
735 / 2221
736 / 2221
737 / 2221
738 / 2221
739 / 2221
740 / 2221
741 / 2221
742 / 2221
743 / 2221
744 / 2221
745 / 2221
746 / 2221
747 / 2221
748 / 2221
749 / 2221
750 / 2221
751 / 2221
752 / 2221
753 / 2221
754 / 2221
755 / 2221
756 / 2221
757 / 2221
758 / 2221
759 / 2221
760 / 2221
761 / 2221
762 / 2221
763 / 2221
764 / 2221
765 / 2221
766 / 2221
767 / 2221
768 / 2221
769 / 2221
770 / 2221
771 / 2221
772 / 2221
773 / 2221
774 / 2221
775 / 2221
776 / 2221
777 / 2221
778 / 2221
779 / 2221
780 / 2221
781 / 2221
782 / 2221
783 / 2221
784 / 2221
785 / 2221
786 / 2221
787 / 2221
788 / 2221
789 / 2221
790 / 2221
791 / 2221
792 / 2221
793 / 2221
794 / 2221
795 / 2221
796 / 2221
797 / 2221
798 / 2221
799 / 2221
800 / 2221
801 / 2221
802 / 2221
803 / 2221
PONGGOL SEVENTEENTH AVENUE, 803 / 2221
803 / 2221
804 / 2221
805 / 2221
806 / 2221
807 / 2221
808 / 2221
809 / 2221
810 / 2221
811 / 2221
812 / 2221
813 / 2221
814 

1413 / 2221
1414 / 2221
1415 / 2221
1416 / 2221
1417 / 2221
1418 / 2221
1419 / 2221
1420 / 2221
1421 / 2221
1422 / 2221
1423 / 2221
1424 / 2221
1425 / 2221
1426 / 2221
1427 / 2221
1428 / 2221
1429 / 2221
1430 / 2221
1431 / 2221
1432 / 2221
1433 / 2221
1434 / 2221
1435 / 2221
1436 / 2221
1437 / 2221
1438 / 2221
1439 / 2221
1440 / 2221
1441 / 2221
1442 / 2221
1443 / 2221
1444 / 2221
1445 / 2221
1446 / 2221
1447 / 2221
1448 / 2221
1449 / 2221
1450 / 2221
1451 / 2221
1452 / 2221
1453 / 2221
1454 / 2221
1455 / 2221
1456 / 2221
1457 / 2221
1458 / 2221
1459 / 2221
1460 / 2221
1461 / 2221
1462 / 2221
1463 / 2221
1464 / 2221
1465 / 2221
1466 / 2221
1467 / 2221
1468 / 2221
1469 / 2221
1470 / 2221
1471 / 2221
1472 / 2221
1473 / 2221
1474 / 2221
1475 / 2221
1476 / 2221
1477 / 2221
1478 / 2221
1479 / 2221
1480 / 2221
1481 / 2221
1482 / 2221
1483 / 2221
1484 / 2221
1485 / 2221
1486 / 2221
1487 / 2221
1488 / 2221
1489 / 2221
1490 / 2221
1491 / 2221
1492 / 2221
1493 / 2221
1494 / 2221
1495 / 2221
1496

2093 / 2221
2094 / 2221
2095 / 2221
2096 / 2221
2097 / 2221
2098 / 2221
2099 / 2221
2100 / 2221
2101 / 2221
2102 / 2221
2103 / 2221
2104 / 2221
2105 / 2221
2106 / 2221
2107 / 2221
2108 / 2221
2109 / 2221
2110 / 2221
2111 / 2221
2112 / 2221
2113 / 2221
2114 / 2221
2115 / 2221
2116 / 2221
2117 / 2221
2118 / 2221
2119 / 2221
2120 / 2221
2121 / 2221
2122 / 2221
2123 / 2221
2124 / 2221
2125 / 2221
2126 / 2221
2127 / 2221
2128 / 2221
2129 / 2221
2130 / 2221
2131 / 2221
2132 / 2221
2133 / 2221
2134 / 2221
2135 / 2221
2136 / 2221
2137 / 2221
2138 / 2221
2139 / 2221
2140 / 2221
2141 / 2221
2142 / 2221
2143 / 2221
2144 / 2221
2145 / 2221
2146 / 2221
2147 / 2221
2148 / 2221
2149 / 2221
2150 / 2221
2151 / 2221
2152 / 2221
2153 / 2221
2154 / 2221
2155 / 2221
2156 / 2221
2157 / 2221
2158 / 2221
2159 / 2221
2160 / 2221
2161 / 2221
2162 / 2221
2163 / 2221
2164 / 2221
2165 / 2221
2166 / 2221
2167 / 2221
2168 / 2221
2169 / 2221
2170 / 2221
2171 / 2221
2172 / 2221
2173 / 2221
2174 / 2221
2175 / 2221
2176

In [14]:
def get_lat_long_add(row, lat_long_mapping):
    street = row['street']
    try:
        row['latitude'] = lat_long_mapping[street][0]
        row['longitude'] = lat_long_mapping[street][1]
    except:
        row['latitude'], row['longitude'] = None, None
    return row

In [15]:
new_hdb_df = hdb_df.apply(lambda x: get_lat_long_add(x, lat_long_mapping), axis=1)
new_ura_df = ura_df.apply(lambda x: get_lat_long_add(x, lat_long_mapping), axis=1)

In [16]:
# Only for URA Transactions (PRIVATE)
def get_year(row):
    contract_date = str(row['contractDate'])
    row['year'] = int(contract_date[-2:])
    return row

new_ura_df = new_ura_df.apply(lambda x: get_year(x), axis=1)

In [17]:
final_ura_df = new_ura_df[(new_ura_df.year > 17) & (new_ura_df.latitude.notna()) & (new_ura_df.longitude.notna())]
final_hdb_df = new_hdb_df[(new_hdb_df.latitude.notna()) & (new_hdb_df.longitude.notna())]

In [18]:
final_ura_df.to_csv('new_ura.csv')
final_hdb_df.to_csv('new_hdb.csv')

# 2. Load Datasets

(1) URA Dataset

In [19]:
ura_5y_trans = final_ura_df
ura_5y_trans["district"] = pd.to_numeric(ura_5y_trans["district"])
print(ura_5y_trans.shape)
ura_5y_trans.head()

(75486, 19)


C:\Users\zywon\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,x,y,nettPrice,latitude,longitude,year
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,20
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,5,Land,Freehold,NaN,NaN,NaN,1.292047,103.768591,20
4,COVE DRIVE,TURQUOISE,CCR,224,01-05,1,0318,3,3600000,Condominium,4,Strata,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244209,103.827487,18


(2) Ameneties Dataset

In [20]:
amenities_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/ameneties_per_district.csv' 
ameneties_per_district = pd.read_csv(amenities_url)
ameneties_per_district.drop(ameneties_per_district.columns[0], axis=1, inplace=True)
print(ameneties_per_district.shape)
ameneties_per_district.head()

(28, 4)


,district,school,hawkercentre,supermarkets
0,1,"['DUNMAN HIGH SCHOOL', 'QUEENSTOWN SECONDARY S...","['Market Street Interim Hawker Centre', 'Maxwe...",[]
1,2,"['DUNMAN HIGH SCHOOL', 'QUEENSTOWN SECONDARY S...",['Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Pag...,[]
2,3,"['QUEENSTOWN SECONDARY SCHOOL', 'SCHOOL OF SCI...",['Bukit Merah View Blk 115 (Blk 115 Bukit Mera...,"['FISHOP PTE. LTD.', 'NTUC FAIRPRICE CO-OPERAT..."
3,4,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."
4,5,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."


(3) Bus Stops Dataset

In [21]:
# Obtain bus stop data
bus_stops_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/bus_stops.csv' 
bus_stops = pd.read_csv(bus_stops_url)
bus_stops.drop(bus_stops.columns[0], axis=1, inplace=True)
print(bus_stops.shape)
bus_stops.head()

(5021, 5)


,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414


(4.1) Crime Locations Dataset - Overview

In [22]:
crime_locations_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/overview_years_crimes_locations_v3.csv' 
overview_years_crimes_locations_v3 = pd.read_csv(crime_locations_url)
overview_years_crimes_locations_v3.drop(overview_years_crimes_locations_v3.columns[3], axis=1, inplace=True) # dropped 'Number (Cases)' column coz only difference between this and 'Number' is that 'Number' states the 0s
print(overview_years_crimes_locations_v3.shape)
overview_years_crimes_locations_v3.head()

(1680, 6)


,Year,NPC,Offence,Neighbourhood,Postal,Number
0,2012,Central Police Division - Total,Robbery,[Total]Central Police Division - Total,-,34
1,2012,Central Police Division - Total,Housebreaking,[Total]Central Police Division - Total,-,32
2,2012,Central Police Division - Total,Snatch Theft,[Total]Central Police Division - Total,-,31
3,2012,Central Police Division - Total,Theft Of Motor Vehicle,[Total]Central Police Division - Total,-,47
4,2012,Central Police Division - Total,Outrage Of Modesty,[Total]Central Police Division - Total,-,193


(4.2) Crime Locations Dataset - Average

In [23]:
avg_crime_locations_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/average_crimes_by_location_v3.csv' 
average_crimes_by_location_v3 = pd.read_csv(avg_crime_locations_url)
average_crimes_by_location_v3

,Neighbourhood,Postal,Offence,Number
0,Ang Mo Kio North,569784,Housebreaking,4.857143
1,Ang Mo Kio North,569784,Outrage Of Modesty,26.000000
2,Ang Mo Kio North,569784,Robbery,2.000000
3,Ang Mo Kio North,569784,Snatch Theft,2.714286
4,Ang Mo Kio North,569784,Theft Of Motor Vehicle,5.000000
...,...,...,...,...
210,[Total]Woodlands Police Division - Total,-,Housebreaking,0.285714
211,[Total]Woodlands Police Division - Total,-,Outrage Of Modesty,2.142857
212,[Total]Woodlands Police Division - Total,-,Robbery,0.000000
213,[Total]Woodlands Police Division - Total,-,Snatch Theft,0.000000


(5) Resale Flat Prices Dataset

In [24]:
resale_flat_prices = final_hdb_df
print(resale_flat_prices.shape)
resale_flat_prices.head()

(92235, 15)


,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,ANG MO KIO AVE 10,1.367602,103.856844
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,ANG MO KIO AVE 4,1.382319,103.838706
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,ANG MO KIO AVE 5,1.376825,103.847556
3,3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,ANG MO KIO AVE 10,1.367602,103.856844
4,4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,ANG MO KIO AVE 5,1.376825,103.847556


In [25]:
sentiment_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/combined_sentiment.csv' 
sentiment = pd.read_csv(sentiment_url)
sentiment

,district_num,hwz_sentiment,hwz_comment_count,year,reddit_sentiment,reddit_comment_count,total_count,weighted_sentiment
0,21,0.000000,0,2018,0.145982,14,14,0.145982
1,1,0.082516,8,2018,0.079938,676,684,0.079968
2,2,0.317045,1,2018,0.087518,23,24,0.097082
3,3,0.123678,39,2018,0.055071,571,610,0.059457
4,4,0.093259,30,2018,0.051841,813,843,0.053315
...,...,...,...,...,...,...,...,...
107,24,0.150795,21,2021,0.077776,642,663,0.080089
108,25,0.088860,12,2021,0.043112,239,251,0.045299
109,26,-0.025054,1,2021,0.000000,0,1,-0.025054
110,27,0.084183,45,2021,0.031414,319,364,0.037938


# 3 BIG DATA SETS

## 3.1. URA
    - ameneties_in_district = ura_5y_trans + ameneties_per_district
    - ameneties_in_district = ameneties_in_district + bus_stops
    - ameneties_and_crimes_in_district = ameneties_in_district + average_crimes_by_location_v3
            
## 3.2. RESALE FLAT PRICES
    - resale_flats_in_district = resale_flat_prices + ameneties_per_district
    - resale_flats_in_district = resale_flats_in_district + bus_stops 
    - resale_flats_and_crime_in_district = resale_flats_in_district + average_crimes_by_location_v3
            
## Append resale_flats_and_crime_in_district to ameneties_and_crimes_in_district
    1. district
    2. street
    3. propertyType (add [propertyType]=Flat in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number

## 3.1 Combine URA and ameneties

In [26]:
# combine ura_5y_trans and ameneties_per_district
ameneties_in_district = ura_5y_trans.join(ameneties_per_district.set_index('district'), on='district')
print(ameneties_in_district.shape)
ameneties_in_district.head()

(75486, 22)


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,tenure,x,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,...,Freehold,NaN,NaN,NaN,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
4,COVE DRIVE,TURQUOISE,CCR,224,01-05,1,0318,3,3600000,Condominium,...,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244209,103.827487,18,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."


## 3.1.1 Combine Combined URA (ameneties_in_district) and Bus Stops

Calculate number of bus stops near each listing

In [27]:
# Formula to calculate distance 
from math import cos, sqrt
R = 6371000 #radius of the Earth in m
def distance(lon1, lat1, lon2, lat2):
    x = (lon2 - lon1) * cos(0.5*(lat2+lat1))
    y = (lat2 - lat1)
    return R * sqrt( x*x + y*y )

In [28]:
busStops = bus_stops.to_dict(orient='records')

In [29]:
# threshold of within 1km
numOfStops = []
busStop_names = [] # to check
for long, lat in zip(ameneties_in_district.longitude, ameneties_in_district.latitude):
    buslist = list(filter(lambda d: distance(d["Longitude"], d["Latitude"], long, lat) <= 1000, busStops))
    busStop_names.append(buslist) # to check
    numOfStops.append(len(buslist))

In [30]:
ameneties_in_district['Bus Stops Nearby'] = numOfStops
ameneties_in_district['Bus Stop Names'] = busStop_names

In [31]:
ameneties_in_district.head()

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets,Bus Stops Nearby,Bus Stop Names
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,...,NaN,NaN,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,...,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,...,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,...,NaN,NaN,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,"[{'BusStopCode': 16081, 'RoadName': 'Pasir Pan..."
4,COVE DRIVE,TURQUOISE,CCR,224,01-05,1,0318,3,3600000,Condominium,...,25008.33592,NaN,1.244209,103.827487,18,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,[]


In [32]:
# check bus stops: Neo Pee Teck Lane
print("longitude: ",ameneties_in_district['longitude'][3])
print("latitude: ",ameneties_in_district['latitude'][3])
print("no. bus stops: ",ameneties_in_district['Bus Stops Nearby'][3])
print("name of bus stops: ",ameneties_in_district['Bus Stop Names'][3])

print()
bus_stops

longitude:  103.7685915
latitude:  1.2920468
no. bus stops:  2
name of bus stops:  [{'BusStopCode': 16081, 'RoadName': 'Pasir Panjang Rd', 'Description': 'Aft Palm Green Condo', 'Latitude': 1.29195777797363, 'Longitude': 103.76866138900289}, {'BusStopCode': 16089, 'RoadName': 'Pasir Panjang Rd', 'Description': 'Aft Clementi Rd', 'Latitude': 1.29201138346787, 'Longitude': 103.7690842071612}]



,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414
...,...,...,...,...,...
5016,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5017,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5018,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5019,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [33]:
# check bus stops: Yishun Central 1
print("longitude: ",ameneties_in_district['longitude'][73170])
print("latitude: ",ameneties_in_district['latitude'][73170])
print("no. bus stops: ",ameneties_in_district['Bus Stops Nearby'][73170])
print("name of bus stops: ",ameneties_in_district['Bus Stop Names'][73170])

print()
bus_stops

longitude:  103.8848358
latitude:  1.389954
no. bus stops:  1
name of bus stops:  [{'BusStopCode': 67329, 'RoadName': 'Sengkang East Ave', 'Description': 'Opp Blk 310B', 'Latitude': 1.39001974230515, 'Longitude': 103.8846016818356}]



,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414
...,...,...,...,...,...
5016,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5017,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5018,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5019,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


## 3.1.2 Combine Combined URA (ameneties_in_district) and average overall crime rates 

Average Crime Rates Method
- Using the average crime rates data, find the average overall crime rates for each district
- Average overall crime rates refer to the sum of the average crime rates for all categories

In [34]:
# MAP AVERAGE_CRIMES_BY_LOCATION_V3 [POSTAL] COLUMN INTO A NEW COLUMN NAMED [DISTRICT]

# swap dictionary mapping direction
postal_district = {k: oldk for oldk, oldv in district_postal.items() for k in oldv}

# create new column to obtain the 1st 2 characters of [Postal]
average_crimes_by_location_v3['postal prefix'] = average_crimes_by_location_v3['Postal'].astype(str).str[0:2]

# map postal to district code
average_crimes_by_location_v3['district']= average_crimes_by_location_v3['postal prefix'].map(postal_district) 

# get overall crime rate in each district
average_crimes_by_location_v3 = average_crimes_by_location_v3.groupby(['district']).agg({'Number':'sum'}).reset_index()
average_crimes_by_location_v3.head()

,district,Number
0,10,35.428571
1,12,60.857143
2,14,167.714286
3,15,45.428571
4,16,130.000000


In [35]:
# convert str to numpy
district_int = average_crimes_by_location_v3.district.astype(int)
district_int = pd.DataFrame(district_int)
average_crimes_by_location_v3['district'] = district_int

In [36]:
# combine ura_5y_trans and ameneties_per_district
ameneties_and_crime_in_district = ameneties_in_district.join(average_crimes_by_location_v3.set_index('district'), on='district')
ameneties_and_crime_in_district = ameneties_and_crime_in_district.reset_index(drop=True)
print(ameneties_and_crime_in_district.shape)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(ameneties_and_crime_in_district.head())

(75486, 25)
              street                     project marketSegment   area  \
0       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR  524.3   
1       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR    308   
2       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR    314   
3  NEO PEE TECK LANE  LANDED HOUSING DEVELOPMENT           RCR  159.3   
4         COVE DRIVE                   TURQUOISE           CCR    224   

  floorRange noOfUnits contractDate typeOfSale    price   propertyType  \
0          -         1         0220          3  5500000  Semi-detached   
1          -         1         0918          3  5000000  Semi-detached   
2          -         1         0618          3  4750000  Semi-detached   
3          -         1         0320          3  2630000        Terrace   
4      01-05         1         0318          3  3600000    Condominium   

   district typeOfArea                             tenure            x  \
0         5       Land        

## 3.1.3 Compute remaining lease using tenure
- num of years used = now - completion date
- remaining = 99 - num of years used

In [37]:
contractDate_datetime = []
for index in range(len(ameneties_and_crime_in_district)):
    contractdate_text = str(ameneties_and_crime_in_district['contractDate'][index])
    contractdate_month = int(contractdate_text[0:-2])
    contractdate_year = int("20" + contractdate_text[-2:])
    contractdate_datetime = datetime.date(contractdate_year,contractdate_month,1)
    contractDate_datetime.append(contractdate_datetime)

In [38]:
remaining_lease_list = []
for index in range(len(ameneties_and_crime_in_district)):
    tenure_text = str(ameneties_and_crime_in_district['tenure'][index])
    if tenure_text == "Freehold":
        remaining_lease = "Freehold"
    elif tenure_text == "NA":
        remaining_lease = "NA"
    elif tenure_text == "99 years leasehold":
        remaining_lease = "99 years leasehold"
    else:
        tenure_text_split = tenure_text.split()
        num_years = int(tenure_text_split[0])
        last_text = tenure_text_split[len(tenure_text_split)-1]
        
        if last_text == "01/11/2017":
            date_list = last_text.split("/")
            completion_year = int(date_list[2]) 
            completion_month = int(date_list[1])
        else:
            completion_year = int(last_text)
            completion_month = 1 # Assume January
            
        completion_date = datetime.date(completion_year,completion_month,1)
        contract_date = contractDate_datetime[index]
        
        num_months_used = (contract_date.year - completion_date.year) * 12 + (contract_date.month - completion_date.month)
        num_months_available = num_years*12
        
        remaining_lease_months_total = num_months_available-num_months_used
        
        # convert remaining lease in months to years and months
        if remaining_lease_months_total < 12:
            remaining_lease_months = remaining_lease_months_total
            remaining_lease = str(remaining_lease_months) + " months"
        else:
            remaining_lease_years = math.floor(remaining_lease_months_total/12)
            remaining_lease_months = remaining_lease_months_total - remaining_lease_years*12
            remaining_lease = str(remaining_lease_years) + " years " + str(remaining_lease_months) + " months"
            
    remaining_lease_list.append(remaining_lease)

In [39]:
ameneties_and_crime_in_district['remaining_lease'] = remaining_lease_list
ameneties_and_crime_in_district.head()

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,latitude,longitude,year,school,hawkercentre,supermarkets,Bus Stops Nearby,Bus Stop Names,Number,remaining_lease
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,...,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,...,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,...,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,...,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,"[{'BusStopCode': 16081, 'RoadName': 'Pasir Pan...",NaN,Freehold
4,COVE DRIVE,TURQUOISE,CCR,224,01-05,1,0318,3,3600000,Condominium,...,1.244209,103.827487,18,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,[],NaN,87 years 10 months


## 3.1.4 Additional Cleaning of Columns

In [40]:
# Represent remaining_lease column with numerical values
remaining_lease_numerical = []
for i in range(len(ameneties_and_crime_in_district)):
    text = ameneties_and_crime_in_district['remaining_lease'][i]
    if text == "Freehold":
        num_years = float(999)
    else:
        text_list = text.split(' ')
        len_text_list = len(text_list)
        if 'leasehold' in text:
            num_years = float(text_list[0])
        else:
            years = float(text_list[0])
            months = float(text_list[2])
            num_years_norounding = years + months/12
            num_years = float("{:.2f}".format(num_years_norounding))        
    remaining_lease_numerical.append(num_years)
ameneties_and_crime_in_district['remaining_lease'] = remaining_lease_numerical

In [41]:
ameneties_and_crime_in_district['remaining_lease'].value_counts()

999.00     21843
97.50       1736
97.00       1600
97.33       1323
97.83       1061
           ...  
975.17         1
805.58         1
14.00          1
9939.75        1
945.50         1
Name: remaining_lease, Length: 982, dtype: int64

In [42]:
ameneties_and_crime_in_district['price'] = pd.to_numeric(ameneties_and_crime_in_district['price'])
ameneties_and_crime_in_district['noOfUnits'] = pd.to_numeric(ameneties_and_crime_in_district['noOfUnits'])
ameneties_and_crime_in_district['typeOfSale'] = pd.to_numeric(ameneties_and_crime_in_district['typeOfSale'])
ameneties_and_crime_in_district['area'] = ameneties_and_crime_in_district['area'].astype(float)
# Clean up floor range column 
ameneties_and_crime_in_district['floorRange'] = ameneties_and_crime_in_district['floorRange'].map(lambda x: x.lstrip('~'))
# For Listings with > 1 Unit make sure its the average
ameneties_and_crime_in_district['price'] = np.where(ameneties_and_crime_in_district['noOfUnits'] > 1,
                                           ameneties_and_crime_in_district['price'] / ameneties_and_crime_in_district['noOfUnits'],
                                           ameneties_and_crime_in_district['price'])

ameneties_and_crime_in_district['area'] = np.where(ameneties_and_crime_in_district['noOfUnits'] > 1,
                                           ameneties_and_crime_in_district['area'] / ameneties_and_crime_in_district['noOfUnits'],
                                           ameneties_and_crime_in_district['area'])
# Filter only resale and subsale units (2 and 3)
ameneties_and_crime_in_district = ameneties_and_crime_in_district[ameneties_and_crime_in_district['typeOfSale'].isin([2, 3])].reset_index(drop=True)
# rename columns
ameneties_and_crime_in_district = ameneties_and_crime_in_district.rename(columns={"Number":"crime_number", "area": "floor_area_sqm", "floorRange": "floor_range"})

## 3.1.5 Rearrange columns
    1. district
    2. street
    3. propertyType (add [propertyType]=Flat in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number
    11. Latitude
    12. Longitude
    13. area
    14. Floor Range
    15. sentiment

In [43]:
# Combine sentiment and dataset
ameneties_and_crime_in_district.year = ameneties_and_crime_in_district.year + 2000

ameneties_and_crime_in_district['disyear'] = ameneties_and_crime_in_district['district'].astype(str) + "_" + ameneties_and_crime_in_district['year'].astype(str)

sentiment['disyear'] = sentiment['district_num'].astype(str) + '_' + sentiment['year'].astype(str)

ameneties_and_crime_in_district['sentiment'] = ameneties_and_crime_in_district['disyear'].map(sentiment.set_index('disyear')['weighted_sentiment'])

In [44]:
ameneties_and_crime_in_district.isnull().any()

street              False
project             False
marketSegment       False
floor_area_sqm      False
floor_range         False
noOfUnits           False
contractDate        False
typeOfSale          False
price               False
propertyType        False
district            False
typeOfArea          False
tenure              False
x                    True
y                    True
nettPrice            True
latitude            False
longitude           False
year                False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
Bus Stop Names      False
crime_number         True
remaining_lease     False
disyear             False
sentiment           False
dtype: bool

In [45]:
# drop irrelevant columns
ameneties_and_crime_in_district.columns

Index(['street', 'project', 'marketSegment', 'floor_area_sqm', 'floor_range',
       'noOfUnits', 'contractDate', 'typeOfSale', 'price', 'propertyType',
       'district', 'typeOfArea', 'tenure', 'x', 'y', 'nettPrice', 'latitude',
       'longitude', 'year', 'school', 'hawkercentre', 'supermarkets',
       'Bus Stops Nearby', 'Bus Stop Names', 'crime_number', 'remaining_lease',
       'disyear', 'sentiment'],
      dtype='object')

In [46]:
ameneties_and_crime_in_district = ameneties_and_crime_in_district[['district','street','propertyType','remaining_lease','price',
                                                                   'school','hawkercentre','supermarkets',
                                                                   'Bus Stops Nearby','crime_number', 'latitude', 'longitude',
                                                                   'floor_area_sqm', 'floor_range', 'sentiment']]
ameneties_and_crime_in_district.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,524.3,-,0.129813
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,308.0,-,0.064396
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,314.0,-,0.064396
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,NaN,1.292047,103.768591,159.3,-,0.129813
4,4,COVE DRIVE,Condominium,87.83,3600000.0,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,NaN,1.244209,103.827487,224.0,01-05,0.053315


## 3.2 Combine resale_flat_prices and amenities

In [47]:
# this dictionary shows the mapping from district > 1st 2 characters of postal
district_town = {
  '01': ['Raffles Place', 'Cecil', 'Marina', 'People’s Park'],
  '02': ['Anson', 'Tanjong Pagar'],
  '03': ['Queenstown', 'Tiong Bahru'],
  '04': ['Telok Blangah', 'Harbourfront'],
  '05': ['Pasir Panjang', 'Hong Leong Garden', 'Clementi New Town'], 
  '06': ['High Street', 'Beach Road'],
  '07': ['Middle Road', 'Golden Mile'],
  '08': ['Little India'],
  '09': ['Orchard', 'Cairnhill', 'River Valley'],
  '10': ['Ardmore', 'Bukit Timah', 'Holland Road', 'Tanglin'],
  '11': ['Watten Estate', 'Novena', 'Thomson'],
  '12': ['Balestier', 'Toa Payoh','Serangoon'],
  '13': ['Macpherson', 'Braddell'],
  '14': ['Geylang', 'Eunos'],
  '15': ['Katong', 'Joo Chiat', 'Amber Road'],
  '16': ['Bedok', 'Upper East Coast', 'Eastwood', 'Kew Drive'],
  '17': ['Loyang', 'Changi'],
  '18': ['Tampines', 'Pasir Ris'],
  '19': ['Serangoon Garden', 'Hougang', 'Punggol'],
  '20': ['Bishan', 'Ang Mo Kio'],
  '21': ['Upper Bukit Timah', 'Clementi Park', 'Ulu Pandan'],
  '22': ['Jurong'],
  '23': ['Hillview', 'Dairy Farm', 'Bukit Panjang', 'Choa Chu Kang'],
  '24': ['Lim Chu Kang', 'Tengah'],
  '25': ['Kranji', 'Woodgrove'],
  '26': ['Upper Thomson', 'Springleaf'],
  '27': ['Yishun', 'Sembawang'],
  '28': ['Seletar']
}

# convert all values to upper case
district_town = {key: [ele.upper() for ele in district_town[key] ] for key in district_town }

# swap dictionary mapping direction
town_district = {k: oldk for oldk, oldv in district_town.items() for k in oldv}

# map town to district code
resale_flat_prices['town'].str.upper()  # convert and ensure all town string in the data are in upper case 
resale_flat_prices['district'] = resale_flat_prices['town'].map(town_district)

In [48]:
import requests 
import json

def get_postal_onemap(place, district_postal):
    start_code= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ str("Ang Mo Kio Ave 5")
    s_response = requests.get(start_code)
    s_data = json.loads(s_response.text)
    postal = None
    count = 0
    while postal is None:
        count += 1
        if count == 10:
            postal = None
            break
        for i in range(len(s_data['results'])):
            postal = s_data['results'][i]['POSTAL']
            try:
                postal = int(postal)
            except:
                continue
    for district, sub_dist in district_postal.items():
        if str(postal)[:2] in sub_dist:
            dist = district
    try:
        return dist
    except:
        return -1

In [49]:
# convert nan to -1
resale_flat_prices['district'] = resale_flat_prices['district'].replace(np.nan, -1)
resale_flat_prices.district.unique()

resale_flat_prices['district'] = np.where(resale_flat_prices['district'] == -1,
                                           get_postal_onemap(resale_flat_prices['district'], district_postal),
                                           resale_flat_prices['district'])
# convert str to numpy
resale_district_int = resale_flat_prices.district.astype(int)
resale_district_int = pd.DataFrame(resale_district_int)
resale_flat_prices['district'] = resale_district_int

In [50]:
# Represent remaining_lease column with numerical values
remaining_lease_numerical = []
for i in range(len(resale_flat_prices)):
    text = resale_flat_prices['remaining_lease'][i]
    if text == "Freehold":
        num_years = float('999')
    else:
        text_list = text.split(' ')
        len_text_list = len(text_list)
        if 'leasehold' in text:
            num_years = float(text_list[0])
        else:
            years = float(text_list[0])
            if len(text_list) == 4:
                months = float(text_list[2])
            else:
                months = 0
            num_years_norounding = years + months/12
            num_years = float("{:.2f}".format(num_years_norounding))        
    remaining_lease_numerical.append(num_years)
resale_flat_prices['remaining_lease'] = remaining_lease_numerical

In [51]:
# combine resale_flat_prices and ameneties_per_district
resale_flats_in_district = resale_flat_prices.join(ameneties_per_district.set_index('district'), on='district')
print(resale_flats_in_district.shape)
resale_flats_in_district.head()

(92235, 19)


,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
3,3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
4,4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."


## 3.2.1 Combine Combined resale_flat_prices (resale_flats_in_district) and Bus Stops

In [52]:
# threshold of within 1km
numOfStops = []
for long, lat in zip(resale_flats_in_district.longitude, resale_flats_in_district.latitude):
    buslist = list(filter(lambda d: distance(d["Longitude"], d["Latitude"], long, lat) <= 1000, busStops))
    numOfStops.append(len(buslist))

resale_flats_in_district['Bus Stops Nearby'] = numOfStops
resale_flats_in_district.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets,Bus Stops Nearby
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
3,3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
4,4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0


## 3.2.2 Combine Combined resale_flat_prices (resale_flats_in_district) and average overall crime rates

In [53]:
# combine ura_5y_trans and ameneties_per_district
resale_flats_and_crime_in_district = resale_flats_in_district.join(average_crimes_by_location_v3.set_index('district'), on='district')
print(resale_flats_and_crime_in_district.shape)
resale_flats_and_crime_in_district.head()

(92235, 21)


,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets,Bus Stops Nearby,Number
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,...,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,...,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,...,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
3,3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,...,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
4,4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,...,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286


## 3.2.3 Rearrange columns
    1. district
    2. street
    3. propertyType (add [propertyType]=flat_type in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number
    11. sentiment

In [54]:
def extract_year(x):
    splitted = x.split('-')
    return splitted[0]

resale_flats_and_crime_in_district['year'] = resale_flats_and_crime_in_district.month.apply(extract_year)

resale_flats_and_crime_in_district['disyear'] = resale_flats_and_crime_in_district['district'].astype(str) + '_' + resale_flats_and_crime_in_district['year'].astype(str)

In [55]:
avg_score = ['10_2017', '12_2017', '14_2017', '16_2017', '18_2017', '19_2017','20_2017',
 '23_2017', '27_2017', '3_2017']

In [56]:
for value in avg_score:
    if value == '3_2017':
        curr_district = value[0]
    else:
        curr_district = value[0:2]
    
    next_value = curr_district + '_2018'
    curr_sentiment = sentiment[sentiment.disyear == next_value]['weighted_sentiment'].item()
    
    sentiment = sentiment.append({'district_num': curr_district, 'disyear': value,
                                 'hwz_comment_count': None,'hwz_sentiment': None,
                                 'reddit_comment_count': None, 'reddit_sentiment': None,
                                 'total_count': None, 'weighted_sentiment': curr_sentiment,
                                 'year': 2017}, ignore_index=True)

In [57]:
resale_flats_and_crime_in_district['sentiment'] = resale_flats_and_crime_in_district['disyear'].map(sentiment.set_index('disyear')['weighted_sentiment'])

In [58]:
resale_flats_and_crime_in_district.isnull().any()

Unnamed: 0             False
month                  False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
remaining_lease        False
resale_price           False
street                 False
latitude               False
longitude              False
district               False
school                 False
hawkercentre           False
supermarkets           False
Bus Stops Nearby       False
Number                 False
year                   False
disyear                False
sentiment              False
dtype: bool

In [59]:
# drop irrelevant columns
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.drop(['month', 'town','block', 'street_name', 'flat_model', 'lease_commence_date'], axis = 1) 
resale_flats_and_crime_in_district.columns

Index(['Unnamed: 0', 'flat_type', 'storey_range', 'floor_area_sqm',
       'remaining_lease', 'resale_price', 'street', 'latitude', 'longitude',
       'district', 'school', 'hawkercentre', 'supermarkets',
       'Bus Stops Nearby', 'Number', 'year', 'disyear', 'sentiment'],
      dtype='object')

In [60]:
# add property type
resale_flats_and_crime_in_district['propertyType'] = resale_flats_and_crime_in_district['flat_type']
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.drop(['flat_type'], axis = 1)

# rename resale price as price, Number as crime_number
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.rename(columns={"resale_price": "price","Number":"crime_number", "storey_range": "floor_range"})

In [61]:
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district[['district','street','propertyType','remaining_lease','price',
                                                                         'school','hawkercentre','supermarkets',
                                                                         'Bus Stops Nearby','crime_number', 'latitude',
                                                                         'longitude', 'floor_area_sqm', 'floor_range', 'sentiment']]
resale_flats_and_crime_in_district.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,20,ANG MO KIO AVE 10,2 ROOM,61.33,232000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.367602,103.856844,44.0,10 TO 12,0.056981
1,20,ANG MO KIO AVE 4,3 ROOM,60.58,250000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.382319,103.838706,67.0,01 TO 03,0.056981
2,20,ANG MO KIO AVE 5,3 ROOM,62.42,262000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.376825,103.847556,67.0,01 TO 03,0.056981
3,20,ANG MO KIO AVE 10,3 ROOM,62.08,265000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.367602,103.856844,68.0,04 TO 06,0.056981
4,20,ANG MO KIO AVE 5,3 ROOM,62.42,265000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.376825,103.847556,67.0,01 TO 03,0.056981


## 3.2.4 Align HDB Floors to URA Floors

In [62]:
# Align HDB Floor to Same as URA Data

replace_floors = {
    '01 TO 03' : '01-05',
    '04 TO 06' : '01-05',
    '07 TO 09' : '06-10',
    '10 TO 12' : '11-15',
    '13 TO 15' : '11-15',
    '16 TO 18' : '16-20',
    '19 TO 21' : '16-20',
    '22 TO 24' : '21-25',
    '25 TO 27' : '26-30',
    '28 TO 30' : '26-30',
    '31 TO 33' : '31-35',
    '34 TO 36' : '31-35',
    '37 TO 39' : '36-40',
    '40 TO 42' : '41-45',
    '43 TO 45' : '41-45',
    '46 TO 48' : '46-50',
    
}

resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.replace({'floor_range': replace_floors})

## 3.3 Combine ameneties_and_crime_in_district and resale_flats_and_crime_in_district

In [63]:
df = ameneties_and_crime_in_district.append(resale_flats_and_crime_in_district)
df.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,524.3,-,0.129813
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,308.0,-,0.064396
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,314.0,-,0.064396
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,NaN,1.292047,103.768591,159.3,-,0.129813
4,4,COVE DRIVE,Condominium,87.83,3600000.0,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,NaN,1.244209,103.827487,224.0,01-05,0.053315


In [64]:
df.shape

(133484, 15)

In [65]:
df.columns

Index(['district', 'street', 'propertyType', 'remaining_lease', 'price',
       'school', 'hawkercentre', 'supermarkets', 'Bus Stops Nearby',
       'crime_number', 'latitude', 'longitude', 'floor_area_sqm',
       'floor_range', 'sentiment'],
      dtype='object')

In [66]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number         True
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

## 4. Further cleaning of data

In [67]:
# convert nan to -1
df['crime_number'] = df['crime_number'].replace(np.nan, -1)
df.crime_number.unique()

array([ -1.        , 193.28571429, 200.        ,  35.42857143,
       113.        , 190.71428571, 167.71428571,  60.85714286,
        45.42857143, 202.14285714, 130.        , 122.71428571,
       163.85714286, 169.42857143, 226.        ,  64.        ,
        78.92857143])

In [68]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

In [69]:
# Convert list of schools, hawkers and supermarkets into num of each facility around
# School

import json

num_sch = []
for sch in df['school']:
    curr = sch.strip('][').split(',') 
    if " SINGAPORE'" in curr:
        curr.remove(" SINGAPORE'")
    num_sch.append(len(curr))

df['school'] = num_sch

# Hawker Centre
num_hawker = []
for hawker in df['hawkercentre']:
    curr = hawker.strip('][').split(',')
    num_hawker.append(len(curr))

df['hawkercentre'] = num_hawker

# Supermarkets
num_supermarkets = []
for supermarket in df['supermarkets']:
    curr = supermarket.strip('][').split(',')
    num_supermarkets.append(len(curr))

df['supermarkets'] = num_supermarkets

df

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,27,6,8,0,-1.0,1.282130,103.786879,524.3,-,0.129813
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,27,6,8,0,-1.0,1.282130,103.786879,308.0,-,0.064396
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,27,6,8,0,-1.0,1.282130,103.786879,314.0,-,0.064396
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,27,6,8,2,-1.0,1.292047,103.768591,159.3,-,0.129813
4,4,COVE DRIVE,Condominium,87.83,3600000.0,11,3,2,0,-1.0,1.244209,103.827487,224.0,01-05,0.053315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92230,27,YISHUN AVE 6,EXECUTIVE,66.67,655000.0,18,2,19,0,64.0,1.426142,103.849653,148.0,11-15,0.037938
92231,27,YISHUN RING ROAD,EXECUTIVE,66.83,650000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,11-15,0.037938
92232,27,YISHUN RING ROAD,EXECUTIVE,66.17,607000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,01-05,0.037938
92233,27,YISHUN ST 81,EXECUTIVE,66.00,670000.0,18,2,19,0,64.0,1.415253,103.837021,142.0,01-05,0.037938


In [70]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

## 5. Normalise Data and Split Data

In [71]:
# Label Encoding 
df['street'] = df['street'].astype('category')
df['street'] = df['street'].cat.codes

df['propertyType'] = df['propertyType'].astype('category')
df['propertyType'] = df['propertyType'].cat.codes

df['floor_range'] = df['floor_range'].astype('category')
df['floor_range'] = df['floor_range'].cat.codes

In [72]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'], 
                                                    test_size=0.1,
                                                    random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120135, 14)
(13349, 14)
(120135,)
(13349,)


In [73]:
# Normalise remaining lease for training and test
X_train_lease_mean = X_train['remaining_lease'].mean()

X_train_lease_std = X_train['remaining_lease'].std()
X_train['remaining_lease'] = (X_train['remaining_lease'] - X_train_lease_mean)/X_train_lease_std

X_test['remaining_lease'] = (X_test['remaining_lease'] - X_train_lease_mean)/X_train_lease_std


# Normalise floor area for training and test
X_train_area_mean = X_train['floor_area_sqm'].mean()

X_train_area_std = X_train['floor_area_sqm'].std()
X_train['floor_area_sqm'] = (X_train['floor_area_sqm'] - X_train_area_mean)/X_train_area_std

X_test['floor_area_sqm'] = (X_test['floor_area_sqm'] - X_train_area_mean)/X_train_area_std

In [74]:
y_train = pd.Series(y_train,
              name="price").to_frame()

y_test = pd.Series(y_test,
              name="price").to_frame()

In [75]:
# Normalise price for training and test
y_train_price_mean = y_train['price'].mean()

y_train_price_std = y_train['price'].std()
y_train['price'] = (y_train['price'] - y_train_price_mean)/y_train_price_std

y_test['price'] = (y_test['price'] - y_train_price_mean)/y_train_price_std

In [76]:
print(X_train.shape)
print(X_train.isnull().any())
print()
print(X_test.shape)
print(X_test.isnull().any())
print()
print(y_train.shape)
print(y_train.isnull().any())
print()
print(y_test.shape)
print(y_test.isnull().any())
print()

(120135, 14)
district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

(13349, 14)
district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

(120135, 1)
price    False
dtype: bool

(13349, 1)
price    False
dtype: bool



## 6. Export to csv

In [77]:
# X_train.to_csv("/Users/vickiyew/Documents/BT4222/project/data/training_set.csv", index = False)
# X_test.to_csv("/Users/vickiyew/Documents/BT4222/project/data/testing_set.csv", index = False)
# y_train.to_csv("/Users/vickiyew/Documents/BT4222/project/data/training_labels.csv", index = False)
# y_test.to_csv("/Users/vickiyew/Documents/BT4222/project/data/testing_labels.csv", index = False)
# df.to_csv("/Users/vickiyew/Documents/BT4222/project/data/combined.csv", index = False)

In [78]:
#df.to_csv("/Users/admin/Desktop/Y3S2/BT4222/combined.csv", index = False)

In [79]:
#df.to_csv('combined.csv')